# Fine-tune MedGemma 4B for Nail Disease Clinical Explanations
## Model 2: Clinical Findings → Medical Explanations Pipeline

**Pipeline Architecture:**
- Stage 1 ✅ DONE: MedSigLIP (Image Classification) → "Clubbing" / "Pitting" etc.
- Stage 2 ⭐ NOW: MedGemma 4B (Clinical Explanation) → "What does this mean?"
- Stage 3: MedGemma 27B (Disease Ranking) → "What diseases could cause this?"

Based on: https://github.com/google-health/medgemma
Model: google/medgemma-7b-orcamath-it (Instruction-tuned, 7B params)
License: Apache 2.0

Features: Medical prompt engineering | CSV integration | Overfitting detection | Comprehensive metrics

## Step 1: Setup Environment

In [ ]:
import os
import torch
import json
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

IS_KAGGLE = os.path.exists('/kaggle')
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print('='*60)
print('ENVIRONMENT SETUP')
print('='*60)
print(f'Environment: {"Kaggle" if IS_KAGGLE else "Local/Colab"}')
print(f'Device: {DEVICE}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB')
else:
    print('GPU: None - CPU mode')
print(f'PyTorch: {torch.__version__}')
print('='*60)

In [ ]:
!pip install -q transformers datasets torch bitsandbytes peft trl scikit-learn matplotlib
print('✅ Packages installed')

## Step 2: Import Libraries

In [ ]:
from datasets import Dataset
from transformers import (
    AutoModelForCausalLM, 
    AutoTokenizer, 
    BitsAndBytesConfig,
    TrainingArguments,
    set_seed
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

set_seed(42)
print('✅ Libraries imported')

## Step 3: Load & Explore Dataset (Model 2 Training Data)

In [ ]:
# Load CSV dataset for Model 2: Clinical Explanation Stage
csv_path = '/kaggle/input/nail-disease-medgemma/nail_diseases.csv'

# Find CSV in various possible locations
possible_paths = [
    '/kaggle/input/nail-disease-medgemma/nail_diseases.csv',
    '/kaggle/input/nail-disease-classification/nail_diseases.csv',
    '/kaggle/input/nail-diseases/nail_diseases.csv',
    './nail_diseases.csv'
]

df = None
for path in possible_paths:
    if os.path.exists(path):
        df = pd.read_csv(path)
        csv_path = path
        break

if df is None:
    print(f'❌ CSV file not found in standard locations')
    print('\nAvailable inputs:')
    if IS_KAGGLE:
        for item in os.listdir('/kaggle/input'):
            print(f'  - {item}')
    sys.exit(1)

print(f'✅ Loaded {len(df)} samples from {csv_path}')
print(f'\nDataset Shape: {df.shape}')
print(f'\nColumns: {list(df.columns)}')
print(f'\nFirst row:')
print(df.iloc[0])
print(f'\nData types:')
print(df.dtypes)

## Step 4: Enhanced Medical Prompt Templates for Clinical Explanations

In [ ]:
def create_medical_prompt_model2(row):
    """
    Creates advanced medical prompts for MedGemma Model 2 (Clinical Explanation Stage).
    
    Input: Nail disease classification from Model 1 + clinical findings
    Output: Detailed clinical explanation of findings, differential diagnoses, and systemic implications
    """
    
    # Extract fields (handle missing/None values)
    nail_disease = str(row.get('nail_disease', row.get('disease_name', ''))).strip()
    clinical_findings = str(row.get('clinical_findings', row.get('findings', ''))).strip()
    patient_age = row.get('patient_age', row.get('age', 'unknown'))
    patient_sex = str(row.get('patient_sex', row.get('sex', row.get('gender', 'unknown')))).strip()
    differential_diagnoses = str(row.get('differential_diagnoses', row.get('differentials', ''))).strip()
    systemic_implications = str(row.get('systemic_implications', row.get('implications', ''))).strip()
    treatment_protocol = str(row.get('treatment_protocol', row.get('treatment', ''))).strip()
    
    # Build instruction-following prompt (Orca format)
    prompt = f"""CLINICAL ANALYSIS: Nail Disease Diagnosis

PATIENT DEMOGRAPHICS:
Age: {patient_age}
Sex: {patient_sex}

PRIMARY FINDING (from Model 1 - MedSigLIP):
{nail_disease}

CLINICAL PRESENTATION:
{clinical_findings}

INSTRUCTION:
Based on the nail disease finding and clinical presentation above, provide:
1. Detailed explanation of what the nail finding indicates
2. Possible systemic diseases that could cause this nail finding
3. Recommended diagnostic workup and treatment approach

EXPECTED RESPONSE:
Nail Finding Explanation: {nail_disease} indicates {systemic_implications}

Differential Diagnoses: {differential_diagnoses}

Recommended Treatment: {treatment_protocol}
"""
    
    return prompt.strip()

# Apply prompt template to dataset
df['text'] = df.apply(create_medical_prompt_model2, axis=1)

print(f'✅ Created {len(df)} medical prompts for Model 2 training')
print(f'\nExample prompt (first 500 chars):')
print('='*60)
print(df['text'].iloc[0][:500])
print('='*60)
print(f'\nAverage prompt length: {df["text"].str.len().mean():.0f} chars')
print(f'Max prompt length: {df["text"].str.len().max():.0f} chars')

## Step 5: Data Quality & Validation Check

In [ ]:
# Check for missing values and data quality
print('📊 DATA QUALITY REPORT')
print('='*60)

# Missing values
missing = df.isnull().sum()
if missing.sum() > 0:
    print('\n⚠️  Missing values detected:')
    print(missing[missing > 0])
else:
    print('\n✅ No missing values')

# Check text field quality
empty_texts = (df['text'].str.len() < 50).sum()
if empty_texts > 0:
    print(f'\n⚠️  {empty_texts} prompts are too short (<50 chars)')
else:
    print(f'\n✅ All prompts have sufficient length')

# Disease distribution
if 'nail_disease' in df.columns:
    print(f'\n📋 Disease Distribution:')
    print(df['nail_disease'].value_counts())

print('\n' + '='*60)

## Step 6: Split Dataset (Train/Val/Test)

In [ ]:
# Stratified split: 70% train, 15% val, 15% test
split_key = 'nail_disease' if 'nail_disease' in df.columns else None

train_df, temp_df = train_test_split(
    df, 
    test_size=0.3, 
    random_state=42,
    stratify=df[split_key] if split_key else None
)

val_df, test_df = train_test_split(
    temp_df, 
    test_size=0.5, 
    random_state=42,
    stratify=temp_df[split_key] if split_key else None
)

print('📊 DATASET SPLIT')
print('='*60)
print(f'Train: {len(train_df)} samples ({len(train_df)/len(df)*100:.1f}%)')
print(f'Val:   {len(val_df)} samples ({len(val_df)/len(df)*100:.1f}%)')
print(f'Test:  {len(test_df)} samples ({len(test_df)/len(df)*100:.1f}%)')
print(f'Total: {len(df)} samples')
print('='*60)

## Step 7: Create HuggingFace Datasets

In [ ]:
# Create HuggingFace datasets
train_dataset = Dataset.from_pandas(train_df[['text']])
val_dataset = Dataset.from_pandas(val_df[['text']])
test_dataset = Dataset.from_pandas(test_df[['text']])

print('✅ HuggingFace datasets created')
print(f'  Train: {len(train_dataset)} samples')
print(f'  Val:   {len(val_dataset)} samples')
print(f'  Test:  {len(test_dataset)} samples')

## Step 8: Setup Model & Tokenizer (MedGemma 4B/7B)

In [ ]:
# Model configuration (MedGemma 4B or 7B instruction-tuned)
MODEL_ID = 'google/medgemma-7b-orcamath-it'
# Alternative: MODEL_ID = 'google/medgemma-4b-orcamath-it' for lighter footprint

# 4-bit quantization config (memory efficient)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

print(f'Loading model: {MODEL_ID}')
print('This may take 2-3 minutes...')

try:
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_ID,
        quantization_config=bnb_config,
        device_map='auto',
        trust_remote_code=True,
    )
    tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=True)
    tokenizer.pad_token = tokenizer.eos_token
    
    print(f'✅ Model loaded successfully')
    print(f'   Model size: {sum(p.numel() for p in model.parameters()) / 1e9:.2f}B parameters')
except Exception as e:
    print(f'❌ Error loading model: {str(e)[:100]}')
    print('Make sure you have HuggingFace token set up for gated models')

## Step 9: Configure LoRA (Low-Rank Adaptation)

In [ ]:
# Prepare model for k-bit training
model = prepare_model_for_kbit_training(model)

# LoRA configuration
lora_config = LoraConfig(
    r=16,  # Rank
    lora_alpha=32,  # Alpha scaling
    target_modules=['q_proj', 'v_proj', 'k_proj'],  # Query, Value, Key projections
    lora_dropout=0.05,
    bias='none',
    task_type='CAUSAL_LM'
)

model = get_peft_model(model, lora_config)

# Count trainable parameters
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
total_params = sum(p.numel() for p in model.parameters())

print(f'✅ LoRA configured')
print(f'  Total params: {total_params / 1e9:.2f}B')
print(f'  Trainable: {trainable_params / 1e6:.2f}M ({100*trainable_params/total_params:.3f}%)')

## Step 10: Setup Training Configuration

In [ ]:
# Training arguments (optimized for Model 2: Clinical Explanation)
training_args = TrainingArguments(
    output_dir='./medgemma_nail_disease_model2_finetuned',
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    gradient_accumulation_steps=2,
    learning_rate=2e-4,
    lr_scheduler_type='cosine',
    warmup_steps=100,
    weight_decay=0.01,
    max_steps=500,  # Limit steps for faster training
    max_seq_length=512,
    logging_steps=10,
    eval_steps=50,
    save_steps=50,
    evaluation_strategy='steps',
    save_strategy='steps',
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
    logging_dir='./logs',
    optim='paged_adamw_8bit',  # Memory-efficient optimizer
    seed=42,
    dataloader_pin_memory=True,
)

print('✅ Training configuration ready')
print(f'  Output: ./medgemma_nail_disease_model2_finetuned')
print(f'  Epochs: {training_args.num_train_epochs}')
print(f'  Batch size: {training_args.per_device_train_batch_size}')
print(f'  Learning rate: {training_args.learning_rate}')
print(f'  Max steps: {training_args.max_steps}')

## Step 11: Initialize SFT Trainer

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field='text',
    max_seq_length=512,
)

print('✅ Trainer initialized and ready for training')

## Step 12: 🚀 START TRAINING (30-60 minutes)

In [ ]:
print('\n' + '='*60)
print('🚀 STARTING MODEL 2 TRAINING')
print('Stage: Clinical Explanation Fine-tuning')
print('='*60)

train_result = trainer.train()

print('\n' + '='*60)
print('✅ TRAINING COMPLETE')
print(f'Final Training Loss: {train_result.training_loss:.4f}')
print('='*60)

## Step 13: Evaluate & Save Model

In [ ]:
# Evaluate on test set
eval_results = trainer.evaluate(test_dataset)
print(f'Test Loss: {eval_results.get("eval_loss", 0):.4f}')

# Save model
model.save_pretrained('./medgemma_nail_disease_model2_finetuned')
tokenizer.save_pretrained('./medgemma_nail_disease_model2_finetuned')
print('\n✅ Model saved to ./medgemma_nail_disease_model2_finetuned')

## Step 14: Extract & Visualize Training Metrics

In [ ]:
import pandas as pd

history = {'train_loss': [], 'eval_loss': []}

try:
    from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
    if os.path.exists('./logs'):
        for file in sorted(os.listdir('./logs')):
            if 'events.out.tfevents' in file:
                ea = EventAccumulator(os.path.join('./logs', file))
                ea.Reload()
                for tag in ea.Tags().get('scalars', []):
                    events = ea.Scalars(tag)
                    for e in events:
                        if 'eval' in tag and 'loss' in tag:
                            history['eval_loss'].append(e.value)
                        elif 'loss' in tag and 'eval' not in tag:
                            history['train_loss'].append(e.value)
except Exception as e:
    print(f'Note: Could not extract tensorboard data: {str(e)[:50]}')

print(f'Extracted: {len(history["train_loss"])} train steps, {len(history["eval_loss"])} eval steps')

## Step 15: 📊 Plot Loss Curves & Overfitting Analysis

In [ ]:
train_loss = np.array(history['train_loss']) if history['train_loss'] else np.array([])
eval_loss = np.array(history['eval_loss']) if history['eval_loss'] else np.array([])

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
fig.suptitle('MedGemma Model 2: Training Metrics & Overfitting Detection', fontsize=14, fontweight='bold')

# Plot 1: Training Loss
if len(train_loss) > 0:
    axes[0, 0].plot(train_loss, marker='o', markersize=3, linewidth=2, color='blue')
    axes[0, 0].set_title('Training Loss Progression', fontweight='bold')
    axes[0, 0].set_xlabel('Training Step')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)

# Plot 2: Validation Loss
if len(eval_loss) > 0:
    axes[0, 1].plot(eval_loss, marker='s', markersize=3, linewidth=2, color='orange')
    axes[0, 1].set_title('Validation Loss Progression', fontweight='bold')
    axes[0, 1].set_xlabel('Evaluation Step')
    axes[0, 1].set_ylabel('Loss')
    axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Train vs Eval with Gap
if len(eval_loss) > 0 and len(train_loss) > 0:
    min_len = min(len(train_loss), len(eval_loss))
    train_aligned = train_loss[-min_len:]
    eval_aligned = eval_loss[-min_len:]
    
    axes[1, 0].plot(train_aligned, marker='o', label='Train Loss', linewidth=2)
    axes[1, 0].plot(eval_aligned, marker='s', label='Eval Loss', linewidth=2)
    axes[1, 0].fill_between(range(min_len), train_aligned, eval_aligned, alpha=0.2, color='red', label='Overfitting Gap')
    axes[1, 0].set_title('Loss Gap: Train vs Eval', fontweight='bold')
    axes[1, 0].set_xlabel('Step')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Overfitting Metrics Summary
if len(eval_loss) > 0 and len(train_loss) > 0:
    min_len = min(len(train_loss), len(eval_loss))
    train_aligned = train_loss[-min_len:]
    eval_aligned = eval_loss[-min_len:]
    loss_gap = eval_aligned - train_aligned
    
    avg_gap = np.mean(loss_gap)
    max_gap = np.max(loss_gap)
    
    if avg_gap < 0.01:
        status = 'MINIMAL OVERFITTING'
    elif avg_gap < 0.05:
        status = 'MILD OVERFITTING'
    else:
        status = 'MODERATE-SEVERE OVERFITTING'
    
    metrics_text = f'OVERFITTING ANALYSIS\n\nAvg Loss Gap: {avg_gap:.6f}\nMax Loss Gap: {max_gap:.6f}\n\nStatus: {status}\n\nTrain Loss: {train_aligned[-1]:.6f}\nEval Loss: {eval_aligned[-1]:.6f}\n\nImprovement: {(1-eval_aligned[-1]/eval_aligned[0])*100:.1f}%'
    
    axes[1, 1].text(0.5, 0.5, metrics_text, ha='center', va='center', fontsize=10, family='monospace', bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.7))
    axes[1, 1].axis('off')

plt.tight_layout()
plt.savefig('model2_overfitting_analysis.png', dpi=150, bbox_inches='tight')
plt.show()
print('✅ Overfitting analysis saved to model2_overfitting_analysis.png')

## Step 16: 🔍 Detailed Overfitting Report

In [ ]:
if len(eval_loss) > 0 and len(train_loss) > 0:
    min_len = min(len(train_loss), len(eval_loss))
    train_aligned = train_loss[-min_len:]
    eval_aligned = eval_loss[-min_len:]
    loss_gap = eval_aligned - train_aligned
    
    print('\n' + '='*60)
    print('🔍 OVERFITTING DETECTION ANALYSIS')
    print('='*60)
    
    print(f'\n📊 Loss Gap Statistics:')
    print(f'  Average Gap: {np.mean(loss_gap):.6f}')
    print(f'  Max Gap: {np.max(loss_gap):.6f}')
    print(f'  Min Gap: {np.min(loss_gap):.6f}')
    
    print(f'\n📈 Performance Metrics:')
    print(f'  Final Train Loss: {train_aligned[-1]:.6f}')
    print(f'  Final Eval Loss: {eval_aligned[-1]:.6f}')
    print(f'  Loss Improvement: {(1-eval_aligned[-1]/eval_aligned[0])*100:.1f}%')
    
    if np.mean(loss_gap) < 0.01:
        status = '🟢 MINIMAL OVERFITTING (Excellent!)'
    elif np.mean(loss_gap) < 0.05:
        status = '🟡 MILD OVERFITTING (Good)'
    else:
        status = '🔴 MODERATE-SEVERE OVERFITTING'
    
    print(f'\n✅ Status: {status}')
    print('='*60)

## Step 17: Save Training Summary & Metadata

In [ ]:
summary = {
    'pipeline_stage': 'Model 2 - Clinical Explanation',
    'model': 'google/medgemma-7b-orcamath-it',
    'training_type': 'SFT (Supervised Fine-Tuning) with LoRA',
    'lora_rank': 16,
    'lora_alpha': 32,
    'target_modules': ['q_proj', 'v_proj', 'k_proj'],
    'train_samples': len(train_df),
    'val_samples': len(val_df),
    'test_samples': len(test_df),
    'epochs': 3,
    'batch_size': 4,
    'gradient_accumulation_steps': 2,
    'learning_rate': 2e-4,
    'optimizer': 'paged_adamw_8bit',
    'max_steps': 500,
    'quantization': '4-bit (nf4)',
    'dataset_source': csv_path,
}

if len(eval_loss) > 0 and len(train_loss) > 0:
    min_len = min(len(train_loss), len(eval_loss))
    train_aligned = train_loss[-min_len:]
    eval_aligned = eval_loss[-min_len:]
    loss_gap = eval_aligned - train_aligned
    
    summary.update({
        'final_train_loss': float(train_aligned[-1]),
        'final_eval_loss': float(eval_aligned[-1]),
        'avg_loss_gap': float(np.mean(loss_gap)),
        'max_loss_gap': float(np.max(loss_gap)),
        'loss_improvement_percent': float((1-eval_aligned[-1]/eval_aligned[0])*100),
        'overfitting_status': 'MINIMAL' if np.mean(loss_gap) < 0.01 else 'MILD' if np.mean(loss_gap) < 0.05 else 'MODERATE-SEVERE'
    })

with open('model2_training_summary.json', 'w') as f:
    json.dump(summary, f, indent=2)

print('✅ Training Summary:')
print(json.dumps(summary, indent=2))

## Step 18: Test Inference with Clinical Example

In [ ]:
# Load best model for inference
if os.path.exists('./medgemma_nail_disease_model2_finetuned/adapter_model.bin'):
    model.load_state_dict(torch.load('./medgemma_nail_disease_model2_finetuned/adapter_model.bin', map_location=DEVICE))

# Test with clinical examples
test_cases = [
    """CLINICAL ANALYSIS: Nail Disease Diagnosis

PATIENT DEMOGRAPHICS:
Age: 65
Sex: Female

PRIMARY FINDING (from Model 1 - MedSigLIP):
Clubbing

CLINICAL PRESENTATION:
Convex nail beds, increased angle between nail and cuticle, bulbous fingertips. Patient has chronic cough and dyspnea.

INSTRUCTION:
Based on the nail disease finding and clinical presentation above, provide:
1. Detailed explanation of what the nail finding indicates
2. Possible systemic diseases that could cause this nail finding
3. Recommended diagnostic workup and treatment approach

EXPECTED RESPONSE:
"""
]

print('\n' + '='*60)
print('🔍 TEST INFERENCE: Clinical Explanation')
print('='*60)

for i, test_prompt in enumerate(test_cases, 1):
    print(f'\nTest Case {i}:')
    print('-'*60)
    
    inputs = tokenizer(test_prompt, return_tensors='pt').to(DEVICE)
    outputs = model.generate(**inputs, max_new_tokens=150, do_sample=True, top_p=0.9, temperature=0.7)
    result = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    print(result)
    print('-'*60)

## Step 19: 3-Stage Pipeline Integration Overview

In [ ]:
print('\n' + '='*70)
print('🎯 3-STAGE NAIL DISEASE ANALYSIS PIPELINE')
print('='*70)

print('''
┌─────────────────────────────────────────────────────────────────────┐
│ STAGE 1: IMAGE CLASSIFICATION (MedSigLIP - COMPLETED)              │
│ Input:  Nail photograph                                             │
│ Output: Disease class ("Clubbing", "Pitting", "Splinter", etc.)    │
│ Model:  MedSigLIP (Vision-Language Model)                          │
└─────────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────────┐
│ STAGE 2: CLINICAL EXPLANATION (MedGemma 4B - NOW TRAINED!)        │
│ Input:  Disease class + clinical findings + patient data           │
│ Output: Detailed medical explanation & differential diagnoses      │
│ Model:  MedGemma 4B/7B (Fine-tuned with your CSV data)             │
│                                                                     │
│ Example Output:                                                     │
│ "Clubbing indicates increased tissue proliferation at nail bases    │
│  Often caused by chronic lung disease, cardiac disease, or IBD.    │
│  Recommend: CXR, ECG, pulmonary function tests..."                 │
└─────────────────────────────────────────────────────────────────────┘
                              ↓
┌─────────────────────────────────────────────────────────────────────┐
│ STAGE 3: DISEASE RANKING (MedGemma 27B - NEXT)                     │
│ Input:  Clinical explanation + patient history                     │
│ Output: Ranked list of probable diseases + urgency levels          │
│ Model:  MedGemma 27B (More capable for complex reasoning)          │
│                                                                     │
│ Example Output:                                                     │
│ 1. Lung Cancer (High urgency) - requires immediate CT scan         │
│ 2. COPD (Medium) - needs pulmonary function tests                  │
│ 3. Heart Disease (Medium) - needs echocardiogram                   │
└─────────────────────────────────────────────────────────────────────┘
''')

print('\n✅ MODEL 2 TRAINING COMPLETE!')
print('\n📁 Output Files:')
print('  ✅ medgemma_nail_disease_model2_finetuned/')
print('     - adapter_model.bin (LoRA weights)')
print('     - config.json')
print('     - tokenizer files')
print('  ✅ model2_overfitting_analysis.png')
print('  ✅ model2_training_summary.json')
print('\n🚀 Next Steps:')
print('  1. Download all files from Kaggle Output tab')
print('  2. Integrate Model 2 with Stage 1 (MedSigLIP)')
print('  3. Start Stage 3 training with MedGemma 27B')
print('  4. Build 3-screen mobile/web app for complete pipeline')
print('\n' + '='*70)

## Step 20: Save Pipeline Integration Code

In [ ]:
# Save integration code snippet for 3-stage pipeline
integration_code = '''# INTEGRATION: 3-Stage Nail Disease Analysis Pipeline
# Use this code to combine all three stages

from PIL import Image
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

class NailDiseaseAnalyzer:
    def __init__(self, model1_path, model2_path, model3_path):
        # Stage 1: MedSigLIP (Image Classification)
        self.model1 = load_medsiglip(model1_path)
        
        # Stage 2: MedGemma 4B (Clinical Explanation)
        self.model2 = AutoModelForCausalLM.from_pretrained("google/medgemma-7b-orcamath-it")
        self.model2 = PeftModel.from_pretrained(self.model2, model2_path)
        
        # Stage 3: MedGemma 27B (Disease Ranking)
        self.model3 = AutoModelForCausalLM.from_pretrained("google/medgemma-27b-orcamath-it")
        self.model3 = PeftModel.from_pretrained(self.model3, model3_path)
        
        self.tokenizer2 = AutoTokenizer.from_pretrained("google/medgemma-7b-orcamath-it")
        self.tokenizer3 = AutoTokenizer.from_pretrained("google/medgemma-27b-orcamath-it")
    
    def analyze_nail_image(self, image_path, patient_age, patient_sex, clinical_findings):
        # Stage 1: Classify nail disease from image
        nail_disease = self.model1.predict(Image.open(image_path))
        
        # Stage 2: Generate clinical explanation
        prompt2 = f"""CLINICAL ANALYSIS: Nail Disease Diagnosis
        
        PATIENT DEMOGRAPHICS:
        Age: {patient_age}
        Sex: {patient_sex}
        
        PRIMARY FINDING:
        {nail_disease}
        
        CLINICAL PRESENTATION:
        {clinical_findings}
        
        INSTRUCTION: Provide detailed explanation..."""
        
        explanation = self.model2.generate(self.tokenizer2(prompt2, return_tensors='pt'))
        
        # Stage 3: Rank possible diseases
        prompt3 = f"""Based on the nail disease {nail_disease} and explanation: {explanation}
        
        Rank the most probable systemic diseases..."""
        
        diagnosis_ranking = self.model3.generate(self.tokenizer3(prompt3, return_tensors='pt'))
        
        return {
            "stage1_classification": nail_disease,
            "stage2_explanation": explanation,
            "stage3_diagnosis_ranking": diagnosis_ranking
        }

# Usage:
analyzer = NailDiseaseAnalyzer(
    model1_path="./medsiglib_nail_classifier",
    model2_path="./medgemma_nail_disease_model2_finetuned",
    model3_path="./medgemma_nail_disease_model3_finetuned"
)

result = analyzer.analyze_nail_image(
    image_path="nail_photo.jpg",
    patient_age=65,
    patient_sex="Female",
    clinical_findings="Convex nail beds with bulbous fingertips, chronic cough"
)

print(result)
'''

with open('pipeline_integration_code.py', 'w') as f:
    f.write(integration_code)

print('✅ Pipeline integration code saved to pipeline_integration_code.py')
print('\nYou can use this code template to combine all 3 stages!')

## Complete! ✅

In [ ]:
print('\n' + '='*60)
print('✅ MODEL 2 FINE-TUNING & ANALYSIS COMPLETE!')
print('='*60)
print('\n📁 Output Files:')
print('  ✅ medgemma_nail_disease_model2_finetuned/')
print('     - adapter_model.bin (LoRA weights)')
print('     - config.json')
print('     - tokenizer files')
print('  ✅ model2_overfitting_analysis.png (4-subplot visualization)')
print('  ✅ model2_training_summary.json (metrics & config)')
print('  ✅ pipeline_integration_code.py (3-stage template)')
print('  ✅ logs/ (tensorboard data)')
print('\n🚀 Next Steps:')
print('  1. Download files from Kaggle Output tab')
print('  2. Use model2 for clinical explanations in your app')
print('  3. Start Stage 3 training with MedGemma 27B')
print('  4. Build mobile/web app integrating all 3 stages')
print('\n📊 Model Performance:')
if len(eval_loss) > 0:
    print(f'  Final Test Loss: {eval_loss[-1]:.4f}')
print('='*60)